<a href="https://colab.research.google.com/github/Bae-hong-seob/University-3-2/blob/main/Big_data_process_and_application/3_Dataframe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark
!apt install openjdk-8-jdk-headless -qq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 47 kB/s 
     |████████████████████████████████| 199 kB 44.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=ac10c1ffe47d14402a9e5cb20442372ff4944f0816abfbfcde748f9fac49c273
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upg

In [2]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [3]:
import pyspark
from pyspark.sql import *

# 두 줄은 붙어서 작성되어야한다.
import pyspark.sql.functions as functions 
from pyspark.sql.types import *
from pyspark import SparkContext, SparkConf


In [4]:
sc = pyspark.SparkContext()
spark = SparkSession.builder.getOrCreate()

In [5]:
# strinJSONRDD RDD생성

stringJSONRDD = sc.parallelize(('''{'id':'123', 'name':'katie', 'age':19,'eyeColor':'brown'}''', 
                                '''{'id':'234', 'name':'Michael', 'age':22,'eyeColor':'green'}''',
                                '''{'id':'345', 'name':'Simone', 'age':23,'eyeColor':'blue'}'''))

In [6]:
# RDD를 Dataframe 객체로 변환
swimmersJSON = spark.read.json(stringJSONRDD)

In [7]:
swimmersJSON.show()

+---+--------+---+-------+
|age|eyeColor| id|   name|
+---+--------+---+-------+
| 19|   brown|123|  katie|
| 22|   green|234|Michael|
| 23|    blue|345| Simone|
+---+--------+---+-------+



ㄴ RDD 형태를 Dataframe 형태로 변환하여 출력함. show()함수 사용

In [8]:
# SQL 쿼리를 통해 데이터 확인.
swimmersJSON.createOrReplaceTempView('swimmersJSON')
print(spark.sql('select * from swimmersJSON where age>20').collect())

[Row(age=22, eyeColor='green', id='234', name='Michael'), Row(age=23, eyeColor='blue', id='345', name='Simone')]


ㄴ select는 레코드 추출. 따라서 조건문은 where절에 삽입

In [9]:
print('---------------------------------------')
spark.sql('select age>20 from swimmersJSON').collect()

---------------------------------------


[Row((age > 20)=False), Row((age > 20)=True), Row((age > 20)=True)]

In [10]:
swimmersJSON.printSchema()

root
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)



ㄴ 구조-- 칼럼명: 자료형 (Null가능성)

# 스키마 변경하기 (직접 만들기)

In [11]:
stringCSVRDD = sc.parallelize([(123,'Katie',19,'brown'),
                               (234,'Michael', 22, 'green'),
                               (345, 'Simone', 23, 'blue')])

In [12]:
# 스키마 명시(=지정)
# 세번째 parameter(=null 값 허용여부) True: Null값이 들어와도 ok
schema = StructType([
    StructField('id', LongType(), True),
    StructField('name', StringType(), True),
    StructField('age', LongType(), True),
    StructField('eyeColor', StringType(), True)
])

In [13]:
# 데이터만 존재하는 stringCSVRDD에 지정한 스키마(shcema) 적용
swimmers = spark.createDataFrame(stringCSVRDD, schema)

In [14]:
# shcema 적용된거 확인
swimmers.printSchema()

root
 |-- id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- eyeColor: string (nullable = true)



# DataFrame 다뤄보기

In [15]:
# 행의 개수 출력
swimmers.createOrReplaceTempView('swimmers')
spark.sql('select count(1) from swimmers').show()

+--------+
|count(1)|
+--------+
|       3|
+--------+



In [16]:
# 필터문(=dataframe) 사용하기 sql where절이랑 똑같음
swimmers.select('id','age').filter('age=22').show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [17]:
spark.sql('select id, age from swimmers where age=22').show()

+---+---+
| id|age|
+---+---+
|234| 22|
+---+---+



In [18]:
# filter(조건문) 사용2
swimmers.select('name','eyeColor').filter("eyeColor like 'b%'").show() #이때 " '' " 큰따음표, 작은따음표로 어디까지 문자열인지 구분해줘야함

+------+--------+
|  name|eyeColor|
+------+--------+
| Katie|   brown|
|Simone|    blue|
+------+--------+



In [19]:
spark.sql('select name, eyeColor from swimmers where eyeColor like "b%"').show()

+------+--------+
|  name|eyeColor|
+------+--------+
| Katie|   brown|
|Simone|    blue|
+------+--------+



# 9/28

# 데이터 전처리 = cleaning 작업

**중복값 제거**
1. .distinct() 
- 중복값 제거 (row의 전체 값이 같을 경우 삭제)

2. .dropDuplicates()
- 중복값 제거. but .distinct()와는 다르게 열을 선택하여 열에서만 제거 가능.
- 제일 위에 있는거만 남는다.
- but 파티션이 나누어져 있는 경우는 알 수 없음


In [20]:
members = sc.parallelize([(1, 44.5, 159.9, 33, 'M'),
                          (2, 66.2, 167.2, 45, 'M'),
                          (3, 66.2, 172.6, 23, 'F'),
                          (4, 42.5, 160.5, 33, 'M'),
                          (5, 50.5, 147.2, 54, 'F'),
                          (3, 66.2, 172.6, 23, 'F'),
                          (6, 60.3, 171.2, 25, 'M'),
                          ])

In [21]:
schema = StructType([
    StructField('id', LongType(), True),
    StructField('weight', FloatType(), True),
    StructField('height', FloatType(), True),
    StructField('age', LongType(), True),
    StructField('gender', StringType(), True),
])

members = spark.createDataFrame(members, schema)

In [22]:
members.show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  1|  44.5| 159.9| 33|     M|
|  2|  66.2| 167.2| 45|     M|
|  3|  66.2| 172.6| 23|     F|
|  4|  42.5| 160.5| 33|     M|
|  5|  50.5| 147.2| 54|     F|
|  3|  66.2| 172.6| 23|     F|
|  6|  60.3| 171.2| 25|     M|
+---+------+------+---+------+



In [25]:
member_1 = members.distinct()
member_1.show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  3|  66.2| 172.6| 23|     F|
|  2|  66.2| 167.2| 45|     M|
|  1|  44.5| 159.9| 33|     M|
|  4|  42.5| 160.5| 33|     M|
|  5|  50.5| 147.2| 54|     F|
|  6|  60.3| 171.2| 25|     M|
+---+------+------+---+------+



In [26]:
member_2 = members.dropDuplicates(['weight'])
member_2.show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  4|  42.5| 160.5| 33|     M|
|  1|  44.5| 159.9| 33|     M|
|  5|  50.5| 147.2| 54|     F|
|  6|  60.3| 171.2| 25|     M|
|  2|  66.2| 167.2| 45|     M|
+---+------+------+---+------+



3. withColumn()
- 새로운 column 추가

4. monotonically_increasing_id()
- 단조증가(monotonical)하는 고유한 숫자값 생성

In [32]:
members.withColumn('new_id', functions.monotonically_increasing_id()).show()

+---+------+------+---+------+----------+
| id|weight|height|age|gender|    new_id|
+---+------+------+---+------+----------+
|  1|  44.5| 159.9| 33|     M|         0|
|  2|  66.2| 167.2| 45|     M|         1|
|  3|  66.2| 172.6| 23|     F|         2|
|  4|  42.5| 160.5| 33|     M|8589934592|
|  5|  50.5| 147.2| 54|     F|8589934593|
|  3|  66.2| 172.6| 23|     F|8589934594|
|  6|  60.3| 171.2| 25|     M|8589934595|
+---+------+------+---+------+----------+



**Null값 처리**

Null값 해결 process
1. 미관찰 값 파악
2. null 많은 column은 제거
3. 나머지 num은 추정값으로 채움. (결측치 대체)




In [28]:
df_miss = spark.createDataFrame([(1, 143.5, 5.6, 28, 'M', 100000),
                                 (2, 167.2, 5.4, 45, 'M', None),
                                 (3, None, 5.2, None, None, None),
                                 (4, 144.5, 5.9, 33, 'M', None),
                                 (5, 133.2, 5.7, 54, 'F', None),
                                 (6, 124.1, 5.2, None, 'F', None),
                                 (7, 129.2, 5.3, 42, 'M', 76000)],
                                ['id', 'height', 'weight', 'age', 'gender', 'income'])
df_miss.show()

+---+------+------+----+------+------+
| id|height|weight| age|gender|income|
+---+------+------+----+------+------+
|  1| 143.5|   5.6|  28|     M|100000|
|  2| 167.2|   5.4|  45|     M|  null|
|  3|  null|   5.2|null|  null|  null|
|  4| 144.5|   5.9|  33|     M|  null|
|  5| 133.2|   5.7|  54|     F|  null|
|  6| 124.1|   5.2|null|     F|  null|
|  7| 129.2|   5.3|  42|     M| 76000|
+---+------+------+----+------+------+



In [ ]:
df_miss.rdd.map(
    lambda row: (row['id'], sum((c==None for c in row))).collect()
)

**column제거**

In [30]:
# df_miss 의 columns 중 income이 아닌 column을 순서대로 c가 배정받는다.
# 따라서 column제거를 select를 통하여 구현
df_miss_no_income = df_miss.select([c for c in df_miss.columns if c!= 'income'])

In [31]:
df_miss_no_income.show()

+---+------+------+----+------+
| id|height|weight| age|gender|
+---+------+------+----+------+
|  1| 143.5|   5.6|  28|     M|
|  2| 167.2|   5.4|  45|     M|
|  3|  null|   5.2|null|  null|
|  4| 144.5|   5.9|  33|     M|
|  5| 133.2|   5.7|  54|     F|
|  6| 124.1|   5.2|null|     F|
|  7| 129.2|   5.3|  42|     M|
+---+------+------+----+------+



**Null 값 제거**
1. agg()
- aggregate().
- 열이나 행에 적용할 함수 입력받음.
2. alias()
- 새로운 이름 할당.
3. fillna()
- 결측치를 filnna(x), x값으로 변경

In [33]:
means = df_miss_no_income.agg(
    *[functions.mean(c).alias(c) for c in df_miss_no_income.columns if c != 'gender']
).toPandas().to_dict('records')[0]

In [34]:
means

{'id': 4.0,
 'height': 140.28333333333333,
 'weight': 5.471428571428571,
 'age': 40.4}

In [35]:
means['gender'] = 'missing'
means

{'id': 4.0,
 'height': 140.28333333333333,
 'weight': 5.471428571428571,
 'age': 40.4,
 'gender': 'missing'}

In [36]:
df_miss_no_income.fillna(means).show()

+---+------------------+------+---+-------+
| id|            height|weight|age| gender|
+---+------------------+------+---+-------+
|  1|             143.5|   5.6| 28|      M|
|  2|             167.2|   5.4| 45|      M|
|  3|140.28333333333333|   5.2| 40|missing|
|  4|             144.5|   5.9| 33|      M|
|  5|             133.2|   5.7| 54|      F|
|  6|             124.1|   5.2| 40|      F|
|  7|             129.2|   5.3| 42|      M|
+---+------------------+------+---+-------+



**이상치(outlider) 처리**

1. 데이터값에 하위/상위 제한선 설정
2. 아웃라이너 확인하기

In [39]:
df_outliers = spark.createDataFrame([(1, 143.5, 5.3, 28),
                                     (2, 154.2, 5.5, 45),
                                     (3, 342.3, 5.1, 99),
                                     (4, 144.5, 5.5, 33),
                                     (5, 133.2, 5.4, 54),
                                     (6, 124.1, 5.1, 21),
                                     (7, 129.2, 5.3, 42)],
                                     ['id', 'weight', 'height', 'age'])
df_outliers.show()

+---+------+------+---+
| id|weight|height|age|
+---+------+------+---+
|  1| 143.5|   5.3| 28|
|  2| 154.2|   5.5| 45|
|  3| 342.3|   5.1| 99|
|  4| 144.5|   5.5| 33|
|  5| 133.2|   5.4| 54|
|  6| 124.1|   5.1| 21|
|  7| 129.2|   5.3| 42|
+---+------+------+---+



1. 데이터값에 하위/상위 제한선 설정
- .approxQuantile(column_name, 0~1사이 값, 허용 가능한 수준의 에러)
- 세번째 인자. 허용 가능한 에러(알파)를 낮출 경우 오래걸림 but 정확하게 계산할 수 있음

In [40]:
cols = ['weight', 'height', 'age']
bounds = {}

for col in cols:
  quantiles = df_outliers.approxQuantile(col, [0.25, 0.75], 0.05)
  IQR = quantiles[1] = quantiles[0] #각 col의 분포를 계산하여 평균은 0.5이니까 Q1 : 0.25, Q3 : 0.75로 인자를 전달하여 IQR을 구함
  bounds[col] = [quantiles[0] - 1.5*IQR, quantiles[1] + 1.5*IQR] # bounds 하위/상위 제한선은 Q1으로 부터 1.5IQR까지 더 내려가도 인정.

bounds

{'weight': [-64.6, 323.0], 'height': [-2.55, 12.75], 'age': [-14.0, 70.0]}

In [41]:
outliers = df_outliers.select(*['id'] + [
    (
        (df_outliers[c] < bounds[c][0]) | # 해당 or 연산자(|) 때문에 값은 True or False로 저장된다.
        (df_outliers[c] > bounds[c][1])
    ).alias(c + '_o') for c in cols
])

outliers.show()

+---+--------+--------+-----+
| id|weight_o|height_o|age_o|
+---+--------+--------+-----+
|  1|   false|   false|false|
|  2|   false|   false|false|
|  3|    true|   false| true|
|  4|   false|   false|false|
|  5|   false|   false|false|
|  6|   false|   false|false|
|  7|   false|   false|false|
+---+--------+--------+-----+

